# Module 10: Final Automation Project

**Difficulty**: ⭐⭐⭐ (Advanced)

**Estimated Time**: 90 minutes

**Prerequisites**: 
- Completed Modules 00-09
- All concepts from previous modules
- Ready to build production-quality automation

## Learning Objectives

By the end of this project, you will have:

1. **Built** a complete end-to-end automation system
2. **Applied** all techniques from modules 00-09
3. **Created** production-ready, deployable code
4. **Demonstrated** professional automation practices
5. **Documented** your system for team use
6. **Deployed** your automation to Windows Task Scheduler

## Project Overview: Data Science Workflow Automator

You'll build a **Data Science Workflow Automation System** that automates a complete data science pipeline.

### The Scenario

You're a data scientist at a company that needs to:
- Collect data from multiple APIs daily
- Process and clean the data
- Generate analysis reports
- Monitor data quality
- Alert team when issues occur
- Archive old data

Currently, this is done manually and takes 2 hours every morning. Your task: **Automate it!**

### System Requirements

**Must Have:**
1. ✅ Data collection from multiple sources
2. ✅ Parallel processing for performance
3. ✅ Data validation and quality checks
4. ✅ Comprehensive logging
5. ✅ Error handling with retries
6. ✅ Email notifications for failures
7. ✅ Secure credential management
8. ✅ Scheduled execution (daily)
9. ✅ Performance monitoring
10. ✅ Automated cleanup of old data

### System Architecture

```
┌─────────────────────────────────────────────────────────┐
│          Data Science Automation System                 │
└─────────────────────────────────────────────────────────┘
                           │
        ┌──────────────────┼──────────────────┐
        │                  │                  │
   ┌────▼────┐      ┌─────▼──────┐   ┌──────▼─────┐
   │ Data    │      │ Processing │   │ Monitoring │
   │Collector│      │  Pipeline  │   │  & Alerts  │
   └────┬────┘      └─────┬──────┘   └──────┬─────┘
        │                 │                  │
        ▼                 ▼                  ▼
   [API Sources]    [Validation]      [Email/Logs]
   [Parallel]       [Transform]       [Metrics]
   [Retry Logic]    [Reports]         [Cleanup]
```

### Technologies Used

- **Module 00-02**: Python environment, subprocess, pathlib
- **Module 03-04**: File operations, process management
- **Module 05**: Environment variables (credentials)
- **Module 06**: HTTP requests, API calls
- **Module 07**: Scheduling, email notifications
- **Module 08**: Security, permissions
- **Module 09**: Parallel processing, logging, testing

In [ ]:
# Setup: Import all required libraries
import os
import sys
import json
import time
import logging
import logging.handlers
from pathlib import Path
from datetime import datetime, timedelta
from concurrent.futures import ThreadPoolExecutor, as_completed
from functools import wraps
import traceback

# Install additional dependencies if needed
import subprocess

try:
    import requests
except ImportError:
    subprocess.run([sys.executable, '-m', 'pip', 'install', 'requests', '-q'])
    import requests

print("✓ All dependencies loaded")
print(f"Python version: {sys.version.split()[0]}")
print(f"Working directory: {Path.cwd()}")
print("\nSystem ready for automation project!")

## Part 1: Configuration and Setup

Professional automation starts with proper configuration management.

In [ ]:
# Configuration system
class AutomationConfig:
    """
    Central configuration for automation system.
    """
    
    def __init__(self, config_file='config.json'):
        self.config_file = Path(config_file)
        self.config = self._load_config()
    
    def _load_config(self):
        """Load configuration from file or create default."""
        if self.config_file.exists():
            with open(self.config_file) as f:
                return json.load(f)
        else:
            # Default configuration
            default_config = {
                'data_dir': 'data/collected',
                'log_dir': 'logs',
                'report_dir': 'reports',
                'archive_dir': 'archive',
                'max_workers': 4,
                'retry_attempts': 3,
                'archive_after_days': 30,
                'data_sources': [
                    {'name': 'api_source_1', 'url': 'https://api.example.com/data1'},
                    {'name': 'api_source_2', 'url': 'https://api.example.com/data2'}
                ],
                'email': {
                    'enabled': False,  # Set to True when configured
                    'smtp_server': os.getenv('SMTP_SERVER', ''),
                    'smtp_port': int(os.getenv('SMTP_PORT', 587)),
                    'from_email': os.getenv('FROM_EMAIL', ''),
                    'to_email': os.getenv('TO_EMAIL', '')
                },
                'quality_thresholds': {
                    'min_rows': 10,
                    'max_missing_pct': 0.1
                }
            }
            
            # Save default config
            self._save_config(default_config)
            return default_config
    
    def _save_config(self, config):
        """Save configuration to file."""
        with open(self.config_file, 'w') as f:
            json.dump(config, f, indent=2)
    
    def get(self, key, default=None):
        """Get configuration value."""
        return self.config.get(key, default)
    
    def update(self, key, value):
        """Update configuration value."""
        self.config[key] = value
        self._save_config(self.config)

# Initialize configuration
config = AutomationConfig('automation_config.json')
print("✓ Configuration loaded")
print(f"  Data directory: {config.get('data_dir')}")
print(f"  Max workers: {config.get('max_workers')}")
print(f"  Data sources: {len(config.get('data_sources'))}")

## Part 2: Logging System

Production logging with rotation and multiple handlers.

In [ ]:
# Setup production logger
def setup_logger(name='automation', log_dir=None):
    """
    Setup production-grade logger.
    
    Args:
        name: Logger name
        log_dir: Log directory (from config if None)
    
    Returns:
        logging.Logger: Configured logger
    """
    if log_dir is None:
        log_dir = config.get('log_dir', 'logs')
    
    log_dir = Path(log_dir)
    log_dir.mkdir(parents=True, exist_ok=True)
    
    # Create logger
    logger = logging.getLogger(name)
    logger.setLevel(logging.DEBUG)
    logger.handlers.clear()
    
    # File handler with rotation
    log_file = log_dir / f"{name}.log"
    file_handler = logging.handlers.RotatingFileHandler(
        log_file,
        maxBytes=10*1024*1024,  # 10 MB
        backupCount=5
    )
    file_handler.setLevel(logging.DEBUG)
    
    # Console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    
    # Formatters
    file_formatter = logging.Formatter(
        '%(asctime)s - %(name)s - %(levelname)s - %(funcName)s:%(lineno)d - %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S'
    )
    console_formatter = logging.Formatter(
        '%(levelname)s: %(message)s'
    )
    
    file_handler.setFormatter(file_formatter)
    console_handler.setFormatter(console_formatter)
    
    logger.addHandler(file_handler)
    logger.addHandler(console_handler)
    
    logger.info(f"Logger initialized: {name}")
    logger.info(f"Log file: {log_file}")
    
    return logger

# Initialize logger
logger = setup_logger('ds_automation')
logger.info("Automation system starting...")

## Part 3: Data Collector (Parallel)

Collect data from multiple sources in parallel with error handling.

In [ ]:
# Retry decorator from Module 09
def retry_with_backoff(max_retries=3, base_delay=1, max_delay=60):
    """Retry decorator with exponential backoff."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            delay = base_delay
            
            for attempt in range(1, max_retries + 1):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if attempt == max_retries:
                        raise
                    
                    logger.warning(f"Attempt {attempt}/{max_retries} failed: {e}")
                    logger.info(f"Retrying in {delay}s...")
                    
                    time.sleep(delay)
                    delay = min(delay * 2, max_delay)
        
        return wrapper
    return decorator

class DataCollector:
    """
    Parallel data collector with error handling.
    """
    
    def __init__(self, config, logger):
        self.config = config
        self.logger = logger
        self.data_dir = Path(config.get('data_dir'))
        self.data_dir.mkdir(parents=True, exist_ok=True)
        self.stats = {'success': 0, 'failed': 0, 'total': 0}
    
    @retry_with_backoff(max_retries=3)
    def collect_from_source(self, source):
        """
        Collect data from a single source.
        
        Args:
            source: Source configuration dict
        
        Returns:
            dict: Collection result
        """
        source_name = source['name']
        source_url = source['url']
        
        self.logger.info(f"Collecting data from: {source_name}")
        
        # For demo, simulate API call
        # In production, use: response = requests.get(source_url, timeout=30)
        
        # Simulate data
        data = {
            'source': source_name,
            'timestamp': datetime.now().isoformat(),
            'data': {
                'records': [
                    {'id': i, 'value': i * 10, 'status': 'active'}
                    for i in range(1, 101)
                ]
            }
        }
        
        # Save to file
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"{source_name}_{timestamp}.json"
        filepath = self.data_dir / filename
        
        with open(filepath, 'w') as f:
            json.dump(data, f, indent=2)
        
        self.logger.info(f"✓ Collected {len(data['data']['records'])} records from {source_name}")
        
        return {
            'source': source_name,
            'filepath': str(filepath),
            'records': len(data['data']['records']),
            'status': 'success'
        }
    
    def collect_all(self):
        """
        Collect data from all sources in parallel.
        
        Returns:
            list: Collection results
        """
        sources = self.config.get('data_sources', [])
        max_workers = self.config.get('max_workers', 4)
        
        self.logger.info(f"Starting parallel data collection from {len(sources)} sources")
        
        results = []
        self.stats['total'] = len(sources)
        
        with ThreadPoolExecutor(max_workers=max_workers) as executor:
            future_to_source = {executor.submit(self.collect_from_source, source): source for source in sources}
            
            for future in as_completed(future_to_source):
                source = future_to_source[future]
                try:
                    result = future.result()
                    results.append(result)
                    self.stats['success'] += 1
                except Exception as e:
                    self.logger.error(f"Failed to collect from {source['name']}: {e}")
                    self.stats['failed'] += 1
                    results.append({
                        'source': source['name'],
                        'status': 'failed',
                        'error': str(e)
                    })
        
        self.logger.info(f"Collection complete: {self.stats['success']} success, {self.stats['failed']} failed")
        return results

# Test data collector
collector = DataCollector(config, logger)
collection_results = collector.collect_all()

print(f"\n✓ Data collection completed")
print(f"  Success: {collector.stats['success']}/{collector.stats['total']}")
print(f"  Failed: {collector.stats['failed']}/{collector.stats['total']}")

## Part 4: Data Validator

Validate data quality before processing.

In [ ]:
# Data quality validator
class DataValidator:
    """
    Validate data quality against thresholds.
    """
    
    def __init__(self, config, logger):
        self.config = config
        self.logger = logger
        self.thresholds = config.get('quality_thresholds', {})
    
    def validate_file(self, filepath):
        """
        Validate a data file.
        
        Args:
            filepath: Path to data file
        
        Returns:
            dict: Validation results
        """
        filepath = Path(filepath)
        self.logger.info(f"Validating: {filepath.name}")
        
        try:
            with open(filepath) as f:
                data = json.load(f)
            
            records = data.get('data', {}).get('records', [])
            
            # Check minimum rows
            min_rows = self.thresholds.get('min_rows', 10)
            if len(records) < min_rows:
                raise ValueError(f"Insufficient data: {len(records)} < {min_rows} rows")
            
            # Check for missing data
            total_fields = len(records) * len(records[0]) if records else 0
            missing_count = sum(
                1 for record in records
                for value in record.values()
                if value is None or value == ''
            )
            missing_pct = missing_count / total_fields if total_fields > 0 else 0
            
            max_missing = self.thresholds.get('max_missing_pct', 0.1)
            if missing_pct > max_missing:
                raise ValueError(f"Too much missing data: {missing_pct:.1%} > {max_missing:.1%}")
            
            self.logger.info(f"✓ Validation passed: {len(records)} records, {missing_pct:.1%} missing")
            
            return {
                'filepath': str(filepath),
                'valid': True,
                'records': len(records),
                'missing_pct': missing_pct
            }
        
        except Exception as e:
            self.logger.error(f"✗ Validation failed: {e}")
            return {
                'filepath': str(filepath),
                'valid': False,
                'error': str(e)
            }
    
    def validate_all(self, results):
        """
        Validate all collected data files.
        
        Args:
            results: Collection results
        
        Returns:
            list: Validation results
        """
        validation_results = []
        
        for result in results:
            if result['status'] == 'success':
                filepath = result['filepath']
                validation = self.validate_file(filepath)
                validation_results.append(validation)
        
        valid_count = sum(1 for r in validation_results if r['valid'])
        self.logger.info(f"Validation complete: {valid_count}/{len(validation_results)} valid")
        
        return validation_results

# Test validator
validator = DataValidator(config, logger)
validation_results = validator.validate_all(collection_results)

print(f"\n✓ Data validation completed")
valid_count = sum(1 for r in validation_results if r['valid'])
print(f"  Valid: {valid_count}/{len(validation_results)}")

## Part 5: Report Generator

Generate summary reports of automation run.

In [ ]:
# Report generator
class ReportGenerator:
    """
    Generate automation reports.
    """
    
    def __init__(self, config, logger):
        self.config = config
        self.logger = logger
        self.report_dir = Path(config.get('report_dir', 'reports'))
        self.report_dir.mkdir(parents=True, exist_ok=True)
    
    def generate_report(self, collection_results, validation_results, execution_time):
        """
        Generate summary report.
        
        Args:
            collection_results: Collection results
            validation_results: Validation results
            execution_time: Total execution time
        
        Returns:
            str: Report filepath
        """
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        report_file = self.report_dir / f"automation_report_{timestamp}.txt"
        
        # Calculate statistics
        total_sources = len(collection_results)
        successful_collections = sum(1 for r in collection_results if r['status'] == 'success')
        failed_collections = total_sources - successful_collections
        
        valid_files = sum(1 for r in validation_results if r['valid'])
        invalid_files = len(validation_results) - valid_files
        
        total_records = sum(r.get('records', 0) for r in validation_results if r['valid'])
        
        # Generate report
        report = f"""# Data Science Automation Report

**Generated**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}
**Execution Time**: {execution_time:.2f}s

## Summary

- ✓ **Status**: {'SUCCESS' if failed_collections == 0 and invalid_files == 0 else 'WARNING' if invalid_files > 0 else 'FAILED'}
- **Total Sources**: {total_sources}
- **Total Records Collected**: {total_records:,}

## Data Collection

- ✓ Successful: {successful_collections}/{total_sources}
- ✗ Failed: {failed_collections}/{total_sources}

## Data Validation

- ✓ Valid: {valid_files}/{len(validation_results)}
- ✗ Invalid: {invalid_files}/{len(validation_results)}

## Details

### Collection Results
"""
        
        for result in collection_results:
            status_icon = "✓" if result['status'] == 'success' else "✗"
            report += f"\n- {status_icon} {result['source']}: {result.get('records', 0)} records"
            if result['status'] == 'failed':
                report += f" (Error: {result.get('error', 'Unknown')})"
        
        report += "\n\n### Validation Results\n"
        
        for result in validation_results:
            status_icon = "✓" if result['valid'] else "✗"
            filepath = Path(result['filepath']).name
            report += f"\n- {status_icon} {filepath}"
            if result['valid']:
                report += f": {result['records']} records, {result['missing_pct']:.1%} missing"
            else:
                report += f": {result.get('error', 'Validation failed')}"
        
        report += "\n\n---\n\nAutomated report generated by Data Science Automation System\n"
        
        # Save report
        with open(report_file, 'w') as f:
            f.write(report)
        
        self.logger.info(f"Report generated: {report_file}")
        
        return str(report_file)

# Generate report
reporter = ReportGenerator(config, logger)
execution_time = 5.0  # Would be measured in production
report_path = reporter.generate_report(collection_results, validation_results, execution_time)

print(f"\n✓ Report generated: {report_path}")

# Display report
with open(report_path) as f:
    print("\n" + "="*60)
    print(f.read())
    print("="*60)

## Part 6: Complete Automation Pipeline

Tie all components together into a single automation system.

In [ ]:
# Main automation pipeline
class DataScienceAutomation:
    """
    Complete data science automation system.
    """
    
    def __init__(self, config_file='automation_config.json'):
        self.config = AutomationConfig(config_file)
        self.logger = setup_logger('ds_automation', self.config.get('log_dir'))
        self.collector = DataCollector(self.config, self.logger)
        self.validator = DataValidator(self.config, self.logger)
        self.reporter = ReportGenerator(self.config, self.logger)
    
    def run(self):
        """
        Execute complete automation pipeline.
        
        Returns:
            dict: Execution results
        """
        start_time = time.time()
        
        try:
            self.logger.info("="*60)
            self.logger.info("Data Science Automation Pipeline Starting")
            self.logger.info("="*60)
            
            # Step 1: Collect data
            self.logger.info("Step 1: Collecting data...")
            collection_results = self.collector.collect_all()
            
            # Step 2: Validate data
            self.logger.info("Step 2: Validating data...")
            validation_results = self.validator.validate_all(collection_results)
            
            # Step 3: Generate report
            execution_time = time.time() - start_time
            self.logger.info("Step 3: Generating report...")
            report_path = self.reporter.generate_report(
                collection_results,
                validation_results,
                execution_time
            )
            
            # Success
            self.logger.info("="*60)
            self.logger.info(f"✓ Pipeline completed successfully in {execution_time:.2f}s")
            self.logger.info("="*60)
            
            return {
                'status': 'success',
                'execution_time': execution_time,
                'collection_results': collection_results,
                'validation_results': validation_results,
                'report_path': report_path
            }
        
        except Exception as e:
            execution_time = time.time() - start_time
            self.logger.error("="*60)
            self.logger.error(f"✗ Pipeline failed: {e}")
            self.logger.error(traceback.format_exc())
            self.logger.error("="*60)
            
            return {
                'status': 'failed',
                'execution_time': execution_time,
                'error': str(e)
            }

# Execute automation
print("\n" + "="*60)
print("Executing Complete Automation Pipeline")
print("="*60 + "\n")

automation = DataScienceAutomation('automation_config.json')
results = automation.run()

if results['status'] == 'success':
    print("\n✓ AUTOMATION SUCCESSFUL")
    print(f"  Execution time: {results['execution_time']:.2f}s")
    print(f"  Report: {results['report_path']}")
else:
    print("\n✗ AUTOMATION FAILED")
    print(f"  Error: {results['error']}")

## Part 7: Deployment Guide

Deploy your automation to run on schedule.

### Deployment Steps

#### 1. Create Standalone Script

Create `run_automation.py`:

```python
#!/usr/bin/env python
"""
Data Science Automation - Standalone Runner
"""
import sys
from pathlib import Path

# Add project directory to path
project_dir = Path(__file__).parent
sys.path.insert(0, str(project_dir))

# Import automation class
from automation import DataScienceAutomation

if __name__ == '__main__':
    automation = DataScienceAutomation('automation_config.json')
    results = automation.run()
    
    # Exit with appropriate code
    sys.exit(0 if results['status'] == 'success' else 1)
```

#### 2. Configure Environment

Create `.env` file (never commit this!):

```env
# Email Configuration
SMTP_SERVER=smtp.gmail.com
SMTP_PORT=587
FROM_EMAIL=your_email@example.com
TO_EMAIL=team@example.com
SMTP_PASSWORD=your_app_specific_password
```

#### 3. Schedule with Task Scheduler

**Option A: Using Python Script**

```python
import subprocess

# Create scheduled task to run daily at 6 AM
task_name = 'DataScienceAutomation'
script_path = r'C:\path\to\run_automation.py'
python_exe = r'C:\path\to\python.exe'

command = [
    'schtasks', '/Create',
    '/TN', task_name,
    '/TR', f'"{python_exe}" "{script_path}"',
    '/SC', 'DAILY',
    '/ST', '06:00',
    '/F'
]

subprocess.run(command)
```

**Option B: Using GUI**

1. Open Task Scheduler (taskschd.msc)
2. Create Basic Task
3. Name: "Data Science Automation"
4. Trigger: Daily at 6:00 AM
5. Action: Start a program
6. Program: `C:\path\to\python.exe`
7. Arguments: `C:\path\to\run_automation.py`
8. Start in: `C:\path\to\project`

#### 4. Test Deployment

```bash
# Test manual run
python run_automation.py

# Check logs
type logs\ds_automation.log

# Check reports
dir reports
```

#### 5. Monitor and Maintain

- Check logs regularly: `logs/ds_automation.log`
- Review reports: `reports/automation_report_*.txt`
- Set up email notifications for failures
- Archive old data monthly
- Update configuration as needed

## Project Summary

### What You Built

✅ **Complete Automation System** with:
1. Parallel data collection from multiple sources
2. Automatic retry logic for failed operations
3. Data quality validation
4. Comprehensive logging with rotation
5. Automated report generation
6. Production-ready error handling
7. Secure configuration management
8. Deployment-ready structure

### Techniques Applied

**From All Modules:**
- Module 00-02: Python environment, subprocess, pathlib
- Module 03: File operations, directory management
- Module 04: Process management
- Module 05: Environment variables
- Module 06: HTTP requests, API integration
- Module 07: Scheduled tasks, automation
- Module 08: Security, permissions
- Module 09: Parallel processing, logging, error handling

### Production Readiness Checklist

Your system includes:

- ✅ Comprehensive error handling
- ✅ Structured logging with rotation
- ✅ Parallel processing for performance
- ✅ Data validation and quality checks
- ✅ Configuration management
- ✅ Retry logic with backoff
- ✅ Report generation
- ✅ Deployment documentation
- ✅ Security best practices

### Next Steps

**To Extend This Project:**

1. **Add Email Notifications**: Implement email alerts using Module 07 techniques
2. **Add More Data Sources**: Expand to real APIs (financial data, weather, etc.)
3. **Add Data Processing**: Include actual data transformation/analysis
4. **Add Database Storage**: Save results to SQLite/PostgreSQL
5. **Add Web Dashboard**: Create Flask app to view reports
6. **Add Testing**: Write unit tests for all components
7. **Add Monitoring**: Integrate with monitoring tools
8. **Add ML Pipeline**: Include model training/prediction

### Congratulations! 🎉

You've completed the Windows for Data Science curriculum and built a production-ready automation system!

**Skills Gained:**
- Windows automation expertise
- Production-grade Python development
- Data pipeline engineering
- System integration
- Professional software practices

**You're now ready to:**
- Automate data science workflows
- Build production data pipelines
- Deploy automated systems
- Optimize Windows-based workflows
- Share your expertise with teams

---

**Keep learning, keep automating!** 🚀